Connected to itmo_dl_course (Python 3.12.8)

In [1]:
import os

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split

import torchvision
import torchvision.transforms as transforms

import pytorch_lightning as pl
from pytorch_lightning.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    TQDMProgressBar,
    LearningRateMonitor,
)
from pytorch_lightning.loggers import TensorBoardLogger

from torchmetrics import Accuracy, F1Score, AUROC, MetricCollection

In [2]:
torch.set_float32_matmul_precision("medium")
pl.seed_everything(42)

 ### Модификация архитектуры AlexNet

 В оригинальной статье AlexNet был разработан для ImageNet с изображениями 227x227x3 и имел следующие особенности:
 1. Разделение сверточных слоев между двумя GPU из-за ограничения в 3GB памяти видеокарт того времени
 2. Использование ReLU как функции активации - впервые для глубоких CNN
 3. Local Response Normalization после первого и второго сверточных слоев
 4. Overlapping pooling с размером ядра 3 и шагом 2
 5. Data augmentation через случайные вырезки 224x224 из 256x256 изображений и отражения по горизонтали
 6. Dropout 0.5 в первых двух полносвязных слоях
 7. Batch size 128
 8. SGD с momentum 0.9 и learning rate, который уменьшался вручную

 Изменения в нашей реализации:

 1. Адаптация архитектуры под CIFAR-100 (32x32x3):
    - Уменьшены размеры ядер и шаги сверточных слоев (kernel_size=3 вместо 11 в первом слое)
    - Изменены размеры входов полносвязных слоев (256 * 4 * 4 вместо 256 * 6 * 6)
    - Уменьшено количество нейронов в полносвязных слоях (1024 вместо 4096)

 2. Модернизация техник нормализации и регуляризации:
    - BatchNormalization вместо Local Response Normalization как более эффективный метод
    - Сохранен Dropout 0.5 в полносвязных слоях
    - Добавлен label smoothing в функцию потерь для лучшей генерализации

 3. Улучшение процесса обучения:
    - OneCycleLR scheduler вместо ручного уменьшения learning rate
    - AdamW оптимизатор вместо SGD для более эффективной оптимизации
    - Упрощенная аугментация данных (только RandomCrop и RandomHorizontalFlip)
    - Gradient clipping для стабильности обучения
    - Mixed precision training для ускорения

 4. Технические изменения:
    - Убрано разделение вычислений между GPU (оригинальное решение было вызвано техническими ограничениями)
    - Добавлен мониторинг метрик через TensorBoard
    - Использован фреймворк PyTorch Lightning для организации кода

In [3]:
class CIFAR100DataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str = "./data", batch_size: int = 128):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

        # Упрощенная аугментация данных
        self.transform_train = transforms.Compose(
            [
                transforms.RandomCrop(32, padding=4),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ]
        )

        self.transform_test = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ]
        )

    def prepare_data(self):
        torchvision.datasets.CIFAR100(self.data_dir, train=True, download=True)
        torchvision.datasets.CIFAR100(self.data_dir, train=False, download=True)

    def setup(self, stage: str):
        if stage == "fit":
            data_full = torchvision.datasets.CIFAR100(
                self.data_dir, train=True, transform=self.transform_train
            )
            train_size = int(0.8 * len(data_full))
            val_size = len(data_full) - train_size
            self.trainset, self.valset = random_split(data_full, [train_size, val_size])

        if stage == "test":
            self.testset = torchvision.datasets.CIFAR100(
                self.data_dir, train=False, transform=self.transform_test
            )

    def train_dataloader(self):
        return DataLoader(
            self.trainset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=os.cpu_count(),
            persistent_workers=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.valset,
            batch_size=self.batch_size,
            num_workers=os.cpu_count(),
            persistent_workers=True,
        )

    def test_dataloader(self):
        return DataLoader(
            self.testset,
            batch_size=self.batch_size,
            num_workers=os.cpu_count(),
            persistent_workers=True,
        )

In [4]:
class ImprovedAlexNet(pl.LightningModule):
    def __init__(self, learning_rate=1e-3, label_smoothing=0.1):
        super().__init__()
        self.save_hyperparameters()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(256 * 4 * 4, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 100),
        )

        self.metrics = MetricCollection(
            [
                Accuracy(task="multiclass", num_classes=100),
                F1Score(task="multiclass", num_classes=100, average="weighted"),
                AUROC(task="multiclass", num_classes=100),
            ]
        )

        self.val_metrics = self.metrics.clone(prefix="val_")
        self.test_metrics = self.metrics.clone(prefix="test_")

        self.criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)

        # Пример входных данных для логирования графа модели
        self.example_input_array = torch.zeros(1, 3, 32, 32)

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        self.log(
            "train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True
        )
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        self.val_metrics.update(logits, y)
        self.log(
            "val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True
        )
        return loss

    def on_validation_epoch_end(self):
        metrics = self.val_metrics.compute()
        self.log_dict(metrics, prog_bar=True, on_epoch=True)
        self.val_metrics.reset()

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        self.test_metrics.update(logits, y)

    def on_test_epoch_end(self):
        metrics = self.test_metrics.compute()
        self.log_dict(metrics, prog_bar=True, on_epoch=True)
        self.test_metrics.reset()

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(), lr=self.hparams.learning_rate, weight_decay=0.0005
        )

        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer,
            max_lr=self.hparams.learning_rate,
            epochs=self.trainer.max_epochs,
            steps_per_epoch=len(self.trainer.datamodule.train_dataloader()),
            pct_start=0.1,
            div_factor=25.0,
            final_div_factor=10000.0,
        )

        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "step",
            },
        }

In [5]:
checkpoint_path = os.path.join("checkpoints", "alexnet_cifar100")
os.makedirs(checkpoint_path, exist_ok=True)

datamodule = CIFAR100DataModule()
model = ImprovedAlexNet()

callbacks = [
    EarlyStopping(monitor="val_loss", mode="min", patience=10, min_delta=1e-4),
    ModelCheckpoint(
        dirpath=checkpoint_path,
        filename="alexnet-cifar100-{epoch:02d}-{val_loss:.2f}",
        save_top_k=3,
        mode="min",
        monitor="val_loss",
    ),
    TQDMProgressBar(refresh_rate=20),
    LearningRateMonitor(logging_interval="step"),
]

logger = TensorBoardLogger(
    save_dir="lightning_logs",
    name="improved_alexnet_cifar100",
    default_hp_metric=False,
)

trainer = pl.Trainer(
    max_epochs=100,
    accelerator="auto",
    devices=1,
    callbacks=callbacks,
    logger=logger,
    gradient_clip_val=1.0,
    precision=16,
    log_every_n_steps=1,
    enable_model_summary=True,
    enable_progress_bar=True,
)

/home/reveur/anaconda3/envs/itmo_dl_course/lib/python3.12/site-packages/lightning_fabric/connector.py:572: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [6]:
trainer.fit(model=model, datamodule=datamodule)

You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


Files already downloaded and verified
Files already downloaded and verified


/home/reveur/anaconda3/envs/itmo_dl_course/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /home/reveur/itmo_dl_course/HW/hw_2/checkpoints/alexnet_cifar100 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Params | Mode  | In sizes       | Out sizes     
--------------------------------------------------------------------------------------------
0 | features     | Sequential       | 2.3 M  | train | [1, 3, 32, 32] | [1, 256, 4, 4]
1 | classifier   | Sequential       | 5.4 M  | train | [1, 4096]      | [1, 100]      
2 | metrics      | MetricCollection | 0      | train | ?              | ?             
3 | val_metrics  | MetricCollection | 0      | train | ?              | ?             
4 | test_metrics | MetricCollection | 0      | train | ?              | ?             
5 | criterion    | CrossEntropyLoss | 0      | train | ?              | ?             
-------------------

/home/reveur/anaconda3/envs/itmo_dl_course/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028


Epoch 57: 100%|██████████| 313/313 [00:32<00:00,  9.55it/s, v_num=2, train_loss_step=1.330, val_loss_step=2.030, val_loss_epoch=2.040, val_MulticlassAccuracy=0.632, val_MulticlassF1Score=0.631, val_MulticlassAUROC=0.978, train_loss_epoch=1.160]


In [7]:
trainer.test(datamodule=datamodule)

/home/reveur/anaconda3/envs/itmo_dl_course/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.


Files already downloaded and verified
Files already downloaded and verified


Restoring states from the checkpoint path at /home/reveur/itmo_dl_course/HW/hw_2/checkpoints/alexnet_cifar100/alexnet-cifar100-epoch=47-val_loss=2.03.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /home/reveur/itmo_dl_course/HW/hw_2/checkpoints/alexnet_cifar100/alexnet-cifar100-epoch=47-val_loss=2.03.ckpt


Testing DataLoader 0: 100%|██████████| 79/79 [00:02<00:00, 31.56it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
  test_MulticlassAUROC      0.9829824566841125
 test_MulticlassAccuracy    0.6600000262260437
 test_MulticlassF1Score     0.6594573259353638
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_MulticlassAccuracy': 0.6600000262260437,
  'test_MulticlassF1Score': 0.6594573259353638,
  'test_MulticlassAUROC': 0.9829824566841125}]

In [8]:
%reload_ext tensorboard
%tensorboard --logdir lightning_logs/